# Importação das Bibliotecas

In [3]:
import cv2
import json
import numpy as np
import os

# Captação das Vagas

In [ ]:
# leitura da imagem
while True:
    entrada = input('Nome do arquivo (com extensão, ex: imagem.png): ') # usuário deve inserir o nome do arquivo
    
    if entrada == 'x':
        print("Encerrado")
        break

    if os.path.isfile(entrada):
        img = cv2.imread(entrada)
        if img is not None:
            break
        else:
            print("Erro ao abrir a imagem. Tente outro arquivo.")
    else:
        print("Arquivo não encontrado. Tenta de novo.")

vaga_atual = []
todas_vagas = []

# função para selecionar as vagas
def clique(event, x, y, flags, param):
    global vaga_atual, todas_vagas

    if event == cv2.EVENT_LBUTTONDOWN: # se o botão esquerdo for pressionado
        vaga_atual.append((x, y)) # adiciona o ponto à lista da vaga atual
        # Desenhar o ponto onde o usuário clicou (círculo azul)
        cv2.circle(img_clone, (x, y), 5, (255, 0, 0), -1)  # desenha um círculo azul onde o usuário clicou (extremidade da vaga)
        print(f"Ponto clicado: {(x, y)}") # printa o ponto clicado

    if len(vaga_atual) == 2: # se o usuário já fez 2 cliques (preencheu uma vaga):
        todas_vagas.append(vaga_atual.copy()) # salva a vaga na lista de vagas, que contém as coordenadas de cada vaga
        cv2.rectangle(img_clone, vaga_atual[0], vaga_atual[1], (255, 0, 0), 2) # desenha um retângulo azul, mostrando onde a vaga foi selecionada
        print(f"Vaga salva: {vaga_atual}") # printa os pontos da vaga, mostrando que ela foi lida
        vaga_atual = [] # reseta a lista da vaga atual para a próxima poder ser armazenada


img = cv2.imread(entrada) # lê a imagem que o usuário digitou
img_clone = img.copy() # cria uma cópia da imagem original, que será usada para desenhar as vagas
cv2.namedWindow("Imagem") # cria uma janela onde a imagem será exibida
cv2.setMouseCallback("Imagem", clique) # define a função 'clique' para ser chamada quando o usuário clicar na janela

while True:
    cv2.imshow("Imagem", img_clone) # mostra a imagem
    key = cv2.waitKey(1)
    if key == 27:
        break # sai do loop se ESC for pressionado

cv2.destroyAllWindows() # fecha a janela aberta pelo opencv

with open("coordenadas_vagas.json", "w") as f:
    json.dump(todas_vagas, f) # salva as coordenadas em um json

Ponto clicado: (551, 185)
Ponto clicado: (657, 296)
Vaga salva: [(551, 185), (657, 296)]
Ponto clicado: (671, 327)
Ponto clicado: (771, 424)
Vaga salva: [(671, 327), (771, 424)]


# Interpretação das Vagas

In [7]:
img = cv2.imread(entrada) # lê a imagem
with open("coordenadas_vagas.json", "r") as f:
    vagas = json.load(f) # lê o json com as coordenadas das vagas definidas

# inicializa as variáveis
livres = 0
ocupadas = 0

def calcular_entropia(imagem):
    histograma = cv2.calcHist([imagem], [0], None, [256], [0, 256]) # calcula o histograma da imagem
    # 0 -> primeiro canal da imagem; None -> não há aplicação de máscara; 256 -> nbins; 0,256 -> intervalo de intensidade
    histograma = histograma / histograma.sum()  # normaliza o histograma dividindo pelo somatório total para garantir que a soma dos valores seja 1
    entropia = -np.sum(histograma * np.log2(histograma + 0.0000001)) # calcula a entropia (evitando log(0))
    return entropia

for i, vaga in enumerate(vagas):
    (x1, y1), (x2, y2) = vaga
    vaga_crop = img[y1:y2, x1:x2]

    cinza = cv2.cvtColor(vaga_crop, cv2.COLOR_BGR2GRAY) # converte a imagem da vaga para a grayscale

    entropia = calcular_entropia(cinza) # calcula a entropia da imagem da vaga em grayscale usando a função criada anteriormente
    print(f"Vaga {i+1}: entropia = {entropia:.2f}") # mostra a entropia da vaga analisada

# definimos o valor 6.5 como sendo adequado para definir a situação das vagas analisadas (valor pode variar conforme imagem analisada)
    if entropia < 6.5: # vaga está livre
        cor = (0, 255, 0) # cor verde -> vaga livre
        livres +=1
    else: # vaga está ocupada
        cor = (0, 0, 255) # cor vermelha -> vaga ocupada
        ocupadas += 1

    cv2.rectangle(img, (x1, y1), (x2, y2), cor, 2) # desenha um retângulo nas coordenadas definidas inicialmente, com a coloração correspondente

cv2.imshow("Resultado", img) # mostra a imagem com as vagas já classificadas
cv2.imwrite("resultado.png", img) # salva a imagem

while True:
    cv2.imshow("Resultado", img) # mostra a imagem
    key = cv2.waitKey(1)
    if key == 27: 
        break # sai do loop se ESC for pressionado

cv2.destroyAllWindows() # fecha a janela aberta pelo opencv


Vaga 1: entropia = 6.95
Vaga 2: entropia = 7.07


# Relatório

In [ ]:
relatorio = []

# escreve no relatório
relatorio.append(f"Total de vagas: {livres + ocupadas}")
relatorio.append(f"Vagas livres: {livres}")
relatorio.append(f"Vagas ocupadas: {ocupadas}")

# salva o relatório
with open("relatorio_vagas.txt", "w") as f:
    f.write("\n".join(relatorio))

# Foto com Legenda

In [ ]:
img_legenda = cv2.imread("resultado.png")

total = livres + ocupadas
texto = [
    f"Total: {total}",
    f"Livres: {livres}",
    f"Ocupadas: {ocupadas}"
]

# fundo branco
overlay = img_legenda.copy()
cv2.rectangle(overlay, (10, img_legenda.shape[0]-50), (220, img_legenda.shape[0]-10), (255, 255, 255), -1)
img_legenda = cv2.addWeighted(overlay, 0.6, img_legenda, 0.4, 0)

# escrever texto com fonte menor
for i, linha in enumerate(texto):
    y = img_legenda.shape[0] - 35 + i*15
    cv2.putText(img_legenda, linha, (20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)

cv2.imwrite("legenda.png", img_legenda)


True